# **Лабораторная работа №1 по вычислительной математикe.**

## Работу выполнил: Шурыгин Антон, Б01 - 909.



In [7]:
import numpy as np

# **Определение основных параметров СЛАУ**




In [28]:
N = 99
SIZE = N + 1

a_vec = np.ones(N-1)
c_vec = np.ones(N)
p_vec = np.full(SIZE, 2)

b_vec = np.array([10 + i for i in range(N)])
f_vec = np.array([i/N for i in range(SIZE)])

- Служебная функция для сравнения с нулем 
- Определение бесконечно малой константы

In [29]:
tolerance = 1e-3

def is_zero(value):
  return value < tolerance

# Определение функций расчета нормы $\|x\|_i$ вектора.

In [30]:
def fst_vec_norm(vec):
  return max(abs(vec))

def sec_vec_norm(vec):
  return abs(max(vec))

def trd_vec_norm(vec):
  return np.sqrt(sum(vec * vec))

vec_norm_list = [fst_vec_norm, sec_vec_norm, trd_vec_norm]

# Определение функций расчета нормы $\|A\|_i$ матрицы.

In [31]:
def fst_matr_norm(matr):
  nrow, ncol = matr.shape
  norm = 0

  for j in range(ncol):
    sum = 0
    for i in range(nrow):
      sum += abs(matr[i][j])
    norm = max(sum, norm)

  return norm

def sec_matr_norm(matr):
  nrow, ncol = matr.shape
  norm = 0

  for i in range(nrow):
    sum = 0
    for j in range(ncol):
      sum += abs(matr[i][j])
    norm = max(sum, norm)

  return norm

def trd_matr_norm(matr):
  return np.sqrt(max(abs(np.linalg.eigvals(np.dot(matr, matr.transpose())))))

matr_norm_list = [fst_matr_norm, sec_matr_norm, trd_matr_norm]

# Функция - инициализация матрицы данной СЛАУ.

In [32]:
def matr_init(size):

  matr = np.zeros((size, size))

  for i in range(size):
    for j in range(size):
      if i == j and i < N and j < N:
        matr[i][j] = 10 + i

      elif i == (j + 1) and i < N:
        matr[i][j] = a_vec[j]
      
      elif j == (i + 1) and j < size:
        matr[i][j] = c_vec[i]

      elif i == size - 1:
        matr[i][j] = p_vec[j]

  return matr

matr = matr_init(SIZE)
correct_sol = np.linalg.solve(matr, f_vec)

# Расчет числа обусловленностей по формуле:

$$
\mu = \|A^{-1}\| \cdot \|A\| 
$$

In [21]:
def mu_calc(norma, matr):
  inv_matr = np.linalg.inv(matr)
  return norma(matr) * norma(inv_matr)

for ind, norm_it in enumerate(matr_norm_list):
  print(f"mu_{ind + 1} = {mu_calc(norm_it, matr)}")

mu_1 = 56.53747551306663
mu_2 = 559.2087436671238
mu_3 = 63.08785084481214


# **Реализация метода Гаусса решения СЛАУ**
- ```gauss_solve_pivot(matr, size)``` - функция оболочка для прямого и обратного хода Гаусса

- ```gauss_pivot_transform(matr, size)``` - реализация алгоритма прямого хода Гаусса с выбором главного элемента

- ```solve_it(matr, size)``` - реализация алгоритма обратного хода Гаусса для получения решения СЛАУ

In [22]:
def gauss_solve_pivot(matr):
  return (solve_it(gauss_pivot_transform(matr, SIZE), SIZE))

def gauss_pivot_transform(matr, size):

    for nrow in range(SIZE):
        pivot = nrow + np.argmax(abs(matr[nrow:, nrow]))

        if pivot != nrow:
            matr[[nrow, pivot]] = matr[[pivot, nrow]]

        row = matr[nrow]
        divisor = row[nrow]

        if is_zero(abs(divisor)):
            raise ValueError("Invalid matrix")

        row /= divisor

        for lower_row in matr[nrow+1:]:
            factor = lower_row[nrow]
            lower_row -= factor*row
    
    return matr

def solve_it(matr, size):
  
    solution = [list for _ in range(size)]
    prev_ind = 0

    for i in range(size):

      cur_ind = size - 1 - i
    
      diag_elem = matr[cur_ind][cur_ind]
      solution[cur_ind] = matr[cur_ind][size] / diag_elem
      
      if (i > 0):
        solution[cur_ind] -= (matr[cur_ind][prev_ind] * (solution[prev_ind])) / diag_elem

      prev_ind = cur_ind
      
    return solution           
 

# **Решение СЛАУ алгоритмом Гаусса**


In [34]:
print("Решение методом Гаусса:")

matr = np.column_stack((matr, f_vec.T))
gauss_sol = gauss_solve_pivot(matr)

print(np.array(gauss_sol))

Решение методом Гаусса:
[-7.93799218e-05  7.93799218e-04  1.44859862e-03  2.02503751e-03
  2.52894404e-03  2.97378631e-03  3.36931185e-03  3.72328469e-03
  4.04191917e-03  4.33025097e-03  4.59240328e-03  4.83178438e-03
  5.05123594e-03  5.25314605e-03  5.43953611e-03  5.61212860e-03
  5.77240032e-03  5.92162466e-03  6.06090546e-03  6.19120427e-03
  6.31336259e-03  6.42812001e-03  6.53612919e-03  6.63796824e-03
  6.73415112e-03  6.82513622e-03  6.91133374e-03  6.99311182e-03
  7.07080180e-03  7.14470272e-03  7.21508515e-03  7.28219449e-03
  7.34625386e-03  7.40746654e-03  7.46601815e-03  7.52207849e-03
  7.57580326e-03  7.62733540e-03  7.67680647e-03  7.72433768e-03
  7.77004093e-03  7.81401968e-03  7.85636971e-03  7.89717985e-03
  7.93653259e-03  7.97450463e-03  8.01116737e-03  8.04658738e-03
  8.08082679e-03  8.11394366e-03  8.14599230e-03  8.17702357e-03
  8.20708516e-03  8.23622182e-03  8.26447559e-03  8.29188600e-03
  8.31849025e-03  8.34432339e-03  8.36941847e-03  8.39380667e-03
 

### Разность векторов метода Гаусса и решения от ```np.linalg.solve(matr, f_vec)```

In [33]:
print(gauss_sol - correct_sol)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.33680869e-19
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  8.67361738e-19 -8.67361738e-19 -8.67361738e-19
  8.67361738e-19  8.67361738e-19 -8.67361738e-19  0.00000000e+00
  8.67361738e-19  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  8.67361738e-19 -8.67361738e-19
  0.00000000e+00  8.67361738e-19  0.00000000e+00  8.67361738e-19
 -8.67361738e-19  8.67361738e-19  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -8.67361738e-19  8.67361738e-19 -8.67361738e-19
  8.67361738e-19  8.67361738e-19  8.67361738e-19  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.73472348e-18
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -1.73472348e-18  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -1.73472348e-18  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

### Из ячейчки выше видно, что полученное решение хорошо совпадает с решением от ```numpy```

# **Расчет невязки для метода Гаусса**

In [234]:
matr = matr_init(SIZE)
err_gauss = f_vec - np.matmul(matr, gauss_sol)

for ind, norm_it in enumerate(vec_norm_list):
  print(f"Невязка метода Гаусса по норме {ind + 1}:", norm_it(err_gauss))

Невязка метода Гаусса по норме 1: 8.881784197001252e-16
Невязка метода Гаусса по норме 2: 2.220446049250313e-16
Невязка метода Гаусса по норме 3: 1.1032241229313589e-15



# **Реализация метода Зейделя решения СЛУ**
```seidel_method_solve(matr, size, norm)``` - реализация итерационного метода Зейделя

```get_LDU(matr, size)``` - разложение исходной матрицы на верхнетреугольную, нижнетреугольную, диагональную



In [235]:
def get_LDU(matr, size):

  L = np.zeros((size, size))
  D = np.zeros((size, size))
  U = np.zeros((size, size)) 

  for i in range(size):
    for j in range(size):
      if i == j:
        D[i][j] = matr[i][j]
      elif i > j:
        L[i][j] = matr[i][j]
      else:
        U[i][j] = matr[i][j]
  
  return L, D, U

def seidel_method_solve(matr, size, norm):

  greater = True
  x = np.zeros(SIZE)

  L, D, U = get_LDU(matr, size)

  LnD_inv = np.linalg.inv(L + D)
  LnD_inv_dotU = np.dot(LnD_inv, U)
  iter_num = 0
  
  while greater:
    
    iter_num += 1
    x_k = -np.dot(LnD_inv_dotU, x)  + np.dot(LnD_inv, f_vec) 
    greater = norm(x_k - x) > tolerance
    x = x_k

  return x, iter_num

# **Решение СЛАУ алгоритмом Зейделя**


In [236]:
matr = matr_init(SIZE)

print("Решение методом Зейделя:")

for ind, norm_it in enumerate(vec_norm_list):
  z_sol, iter_num = seidel_method_solve(matr, SIZE, norm_it)
  print(f"Решение методом Зейделя по норме {ind + 1}:")
  print(z_sol)
  print(f"Сошлось за {iter_num} итераций")
  print(f"Невязка метода Зейделя по норме {ind + 1}:", norm_it(z_sol - cor_sol))

Решение методом Зейделя:
Решение методом Зейделя по норме 1:
[-7.80532599e-05  7.94959555e-04  1.44968973e-03  2.02603641e-03
  2.52985089e-03  2.97460566e-03  3.37005064e-03  3.72395058e-03
  4.04251974e-03  4.33079331e-03  4.59289388e-03  4.83222902e-03
  5.05163978e-03  5.25351362e-03  5.43987140e-03  5.61243511e-03
  5.77268111e-03  5.92188244e-03  6.06114258e-03  6.19142283e-03
  6.31356443e-03  6.42830675e-03  6.53630226e-03  6.63812893e-03
  6.73430055e-03  6.82527541e-03  6.91146358e-03  6.99323312e-03
  7.07091529e-03  7.14480905e-03  7.21518489e-03  7.28228818e-03
  7.34634197e-03  7.40754951e-03  7.46609635e-03  7.52215229e-03
  7.57587297e-03  7.62740133e-03  7.67686888e-03  7.72439681e-03
  7.77009701e-03  7.81407290e-03  7.85642027e-03  7.89722793e-03
  7.93657835e-03  7.97454820e-03  8.01120890e-03  8.04662699e-03
  8.08086460e-03  8.11397977e-03  8.14602682e-03  8.17705658e-03
  8.20711676e-03  8.23625208e-03  8.26450458e-03  8.29191379e-03
  8.31851692e-03  8.34434899e

### Результаты метода Зейделя в сравнении с Гауссом и библиотечным от ```numpy``` так же говорят о корректности реализации